In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

#Vectorizar

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
x_train_text = df_train['review_es']
y_train_text = df_train['sentimiento']

x_test_text = df_test['review_es']

In [ ]:
my_stop_words = stopwords.words('spanish')

# Instanciamos el CV
#Configurar el vectorizador de palabras para convertir el texto en una representación numérica.
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
#transformar los datos de entrenamiento en una matriz numérica.
x_train_aux = vec.fit_transform(x_train_text)
#Transformar los datos de prueba en la misma representación numérica que el conjunto de entrenamiento
x_test = vec.transform(x_test_text)

#Ensamble (Voting)

In [ ]:
from sklearn.ensemble import VotingClassifier
import joblib

In [ ]:
!pip install scikit-learn==1.2.2

##Random forest

In [ ]:
model_rf = joblib.load('/content/drive/MyDrive/modelos/rf_n10cv10.joblib')

##NB

In [ ]:
model_nb = joblib.load('/content/drive/MyDrive/modelos/best_nb.joblib')

##Red Neuronal

In [ ]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.0 which is incompatible.


In [ ]:
def create_model():
    cant_clases = len(np.unique(y_train))
    modelo = keras.Sequential([
        keras.Input(shape=(3192097,)),
        keras.layers.Dense(cant_clases, activation='sigmoid')
    ])
    modelo.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=['f1_score'],
    )
    return modelo

In [ ]:
model_rn = joblib.load('/content/drive/MyDrive/modelos/rn_keras_tf.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator FunctionTransformer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For mor

##Voting con RN, NB y Random Forest

In [ ]:
voting_rn_nb_rf = VotingClassifier(estimators = [('nb', model_nb), ('rn', model_rn), ('rf', model_rf)], voting = 'hard')

In [ ]:
voting_rn_nb_rf.fit(x_train_aux,y_train)

Epoch 1/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 166s 119ms/step - f1_score: 0.4968 - loss: 0.4179
Epoch 2/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 209s 124ms/step - f1_score: 0.5000 - loss: 0.0379
Epoch 3/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 229s 144ms/step - f1_score: 0.4999 - loss: 0.0131
Epoch 4/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 174s 123ms/step - f1_score: 0.5000 - loss: 0.0064
Epoch 5/5
1377/1377 ━━━━━━━━━━━━━━━━━━━━ 209s 129ms/step - f1_score: 0.5000 - loss: 0.0034


VotingClassifier(estimators=[('nb',
                              MultinomialNB(alpha=1.0714285714285714,
                                            class_prior=[0.2, 0.8],
                                            fit_prior=False,
                                            force_alpha=True)),
                             ('rn',
                              KerasClassifier(batch_size=35, epochs=5, model=<function create_model at 0x7f596a883f40>)),
                             ('rf',
                              RandomForestClassifier(criterion='entropy',
                                                     min_samples_leaf=5,
                                                     min_samples_split=14,
                                                     n_estimators=57, n_jobs=-1,
                                                     oob_score=True,
                                                     random_state=1))])

In [ ]:
joblib.dump(voting_rn_nb_rf, '/content/drive/MyDrive/modelos/voting_rn_nb_rf.joblib')

['/content/drive/MyDrive/modelos/voting_rn_nb_rf.joblib']

In [ ]:
voting_rn_nb_rf

VotingClassifier(estimators=[('nb',
                              MultinomialNB(alpha=1.0714285714285714,
                                            class_prior=[0.2, 0.8],
                                            fit_prior=False,
                                            force_alpha=True)),
                             ('rn',
                              KerasClassifier(batch_size=35, epochs=5, model=<function create_model at 0x7f596a883f40>)),
                             ('rf',
                              RandomForestClassifier(criterion='entropy',
                                                     min_samples_leaf=5,
                                                     min_samples_split=14,
                                                     n_estimators=57, n_jobs=-1,
                                                     oob_score=True,
                                                     random_state=1))])

In [ ]:
y_pred_voting_rn_nb_rf = voting_rn_nb_rf.predict(x_test)

246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [ ]:
df_pred_voting = pd.DataFrame()
df_pred_voting['ID'] = df_test['ID']
df_pred_voting['sentimiento'] = y_pred_voting_rn_nb_rf

df_pred_voting.to_csv('/content/drive/MyDrive/pred/y_pred_voting_rn_nb_rf.csv', index=False)